In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

# keras
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

# custom define
from src.load_data import get_read_main
from src.train import create_model_conv, training

os.environ["CUDA_VISIBLE_DEVICES"]="0"

# 讀取資料

In [ ]:
X_train, X_test, y_train, y_test = get_read_main('./characters/', save=False, load=True)
print("Train sample:", X_train.shape[0])
print("Test sample:", X_test.shape[0])

# 建構模型

## compile
在訓練模型之前，您需要配置學習過程，這是通過編譯方法完成的。它接收三個參數：
+ 優化器 optimizer
+ 損失函數 loss
+ 評估指標 metrics

In [ ]:
model = create_model_conv(input_shape=X_train.shape[1:], num_classes=20, show_summary=False)
initial_weights = model.get_weights()
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型

## Callbacks

In [ ]:
checkpoint = ModelCheckpoint('best_weights.hdf5',
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

cb_list = [checkpoint]

In [ ]:
model, history = training(model, 
                          X_train, X_test, y_train, y_test,
                          batch_size=256, epochs=100, 
                          callbacks_list=cb_list)

In [ ]:
# summarize history for accuracy
fig = plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# 比較 Optimizer

In [ ]:
def shuffle_weights(model, weights=None):
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    model.set_weights(weights)

## Run Adam

In [ ]:
shuffle_weights(model, initial_weights)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model, history_Adam = training(model, 
                          X_train, X_test, y_train, y_test,
                          batch_size=256, epochs=100)

## Run SGD

In [ ]:
shuffle_weights(model, initial_weights)
opt = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model, history_SGD = training(model, 
                          X_train, X_test, y_train, y_test,
                          batch_size=256, epochs=100)

In [ ]:
fig = plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.plot(history_Adam.history['acc'])
plt.plot(history_Adam.history['val_acc'])
plt.plot(history_SGD.history['acc'])
plt.plot(history_SGD.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Adam_train', 'Adam_val', 'SGD_train', 'SGD_val'], loc='best')
# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history_Adam.history['loss'])
plt.plot(history_Adam.history['val_loss'])
plt.plot(history_SGD.history['loss'])
plt.plot(history_SGD.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Adam_train', 'Adam_val', 'SGD_train', 'SGD_val'], loc='best')
plt.show()